# k.p example of using the Lowdin perturbation

This example requires [semicon](https://gitlab.kwant-project.org/semicon/semicon) to be installed.

It should be as easy as 
```
pip install git+https://gitlab.kwant-project.org/semicon/semicon.git
```

In [ ]:
try:
    import semicon
except ImportError:
    print("Semicon should be installed to run this notebook.")

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

In [ ]:
import kwant
import semicon

import numpy as np
import scipy.linalg as la
import scipy.sparse

import sympy
sympy.init_printing(print_builtin=False)

import matplotlib.pyplot as plt
%matplotlib inline

# Prepare k.p model and solve it exactly

In [ ]:
widths = [5, 12.5, 5, 5]
gamma_0 = 1.0

grid_spacing = 0.5
shape = lambda site: 0 - grid_spacing / 2 < site.pos[0] < sum(widths)

kpham = semicon.models.foreman('z')

AlSb = semicon.parameters.bulk('lawaetz', 'AlSb', gamma_0, valence_band_offset=.18)
InAs = semicon.parameters.bulk('lawaetz', 'InAs', gamma_0)
GaSb = semicon.parameters.bulk('lawaetz', 'GaSb', gamma_0, valence_band_offset=.56)

params, walls = semicon.parameters.two_deg(
    parameters = [AlSb, InAs, GaSb, AlSb],
    widths = widths,
    grid_spacing=grid_spacing,
    extra_constants=semicon.parameters.constants,
)

In [ ]:
def calc_energy(k):
    p = {'k_x': k, 'k_y': 0}
    mat = syst.hamiltonian_submatrix(params={**p, **params})
    return la.eigvalsh(mat)


template = kwant.continuum.discretize(str(kpham), coords='z', 
                                      grid=grid_spacing)

syst = kwant.Builder()
syst.fill(template, shape, (0, ))
syst = syst.finalized()

N = len(syst.sites)
momenta = np.linspace(-.3, .3, 81)
energies_exact = np.array([calc_energy(k) for k in momenta])

In [ ]:
ylims = (500-5, 530+5)
xlims = (-.3, .3)

window = (.500, .530)

plt.plot(momenta, 1000 * energies_exact, 'k-');
plt.plot(xlims, 1000 * np.array([window, window]))
plt.ylim(*ylims)
plt.xlim(*xlims)

# prepare perturbation basis

In [ ]:
def find_indices(ev, lower, upper):
    return [i for (i, e) in enumerate(ev) if lower < e < upper]

In [ ]:
# for simplicity lets import everything now
from codes.lowdin import *

gens = sympy.symbols(['k_x', 'k_y'])
H0, H1 = prepare_hamiltonian(
    kpham, gens, 'z', grid_spacing, shape, (0,)
)

M = np.diag([1/2, -1/2, 3/2, 1/2, -1/2, -3/2, 1/2, -1/2])
bigM = kwant.operator.Density(H0, M).tocoo().toarray()

## Full diagonalisation: we know everything

In [ ]:
mat0 = H0.hamiltonian_submatrix(params=params)
mat1 = {k: v.hamiltonian_submatrix(params=params) 
        for k, v in H1.items()}

ev, evec = la.eigh(mat0)
indices = find_indices(ev, *window)

U, evs = decouple_basis([bigM, mat0], evec[:, indices])

# ev[indices] = evs[-1]
# evec[:, indices] = evec[:, indices] @ U
# apply_smart_gauge(evec)        

## Old lowdin at higher order

In [ ]:
import codes.old_lowdin as olo

In [ ]:
from importlib import reload
olo = reload(olo)

In [ ]:
gens = sympy.symbols('k_x k_y')
inter_keys = olo.get_maximum_powers(gens, 4)

model = olo.get_effective_model(ev, evec, indices, mat1, inter_keys)

ham = sympy.expand(model.tosympy())

In [ ]:
inter_keys

In [ ]:
model_old = model
ham

## KPM lowdin at higher order

In [ ]:
import codes.higher_order_lowdin as nlo

In [ ]:
from importlib import reload
nlo = reload(nlo)

In [ ]:
kpm_params = dict(num_moments=1000)
gens = sympy.symbols('k_x k_y')
inter_keys = nlo.get_maximum_powers(gens, 4)

evec_A = evec[:, indices].T.conj()
model = nlo.get_effective_model(mat0, mat1, evec_A, inter_keys, order=4, kpm_params=kpm_params)

ham = sympy.expand(model.tosympy(digits=4))

In [ ]:
inter_keys

In [ ]:
model_new = model
ham

In [ ]:
for key in model_old.keys():
    assert np.allclose(model_old[key], model_new[key], atol=1e-2), (key, model_old[key] - model_new[key])

## Old lowdin at higher order

In [ ]:
from importlib import reload
olo = reload(olo)

In [ ]:
# Simple model
ev = np.array([1., 5.])
mat0 = np.diag(ev)
mat1 = {sympy.Symbol('x'): np.array([[0, 1.], [1., 0]])}
evec = np.eye(2)
indices = [0]

In [ ]:
gens = [sympy.symbols('x')]
inter_keys = olo.get_maximum_powers(gens, 2)

model = olo.get_effective_model(ev, evec, indices, mat1, inter_keys)

ham = sympy.expand(model.tosympy())

In [ ]:
inter_keys

In [ ]:
model_old = model
ham

## KPM lowdin at higher order

In [ ]:
import codes.higher_order_lowdin as nlo

In [ ]:
from importlib import reload
nlo = reload(nlo)

In [ ]:
kpm_params = dict(num_moments=1000)
inter_keys = nlo.get_maximum_powers(gens, 2)

model = nlo.get_effective_model(mat0, mat1, evec.T.conj()[indices], inter_keys, order=2, kpm_params=kpm_params)

ham = sympy.expand(model.tosympy(digits=4))

In [ ]:
inter_keys

In [ ]:
model_new = model
ham

In [ ]:
for key in model_old.keys():
    assert np.allclose(model_old[key], model_new[key], atol=1e-2), (key, model_old[key] - model_new[key])

## Sparse diagonalisation: we know subset of states

In [ ]:
# prepare subspace (like input from sparse diag)
indices = find_indices(ev, *window)
subspace_slice = slice(indices[0] - 20, indices[-1] + 20 + 1)
energies = ev[subspace_slice]
subspace = evec[:, subspace_slice]

In [ ]:
# %%time

# import scipy.sparse.linalg as sla

# mat0 = H0.hamiltonian_submatrix(params=params, sparse=True)
# mat1 = {k: v.hamiltonian_submatrix(params=params, sparse=True) 
#         for k, v in H1.items()}

# energies, subspace = sla.eigsh(mat0, sigma=.520, k=10)
# subspace, _ = la.qr(subspace, mode='economic')

# indices = find_indices(energies, *window)

# U, evs = decouple_basis([bigM, mat0], subspace[:, indices])
# energies[indices] = evs[-1]

# subspace[:, indices] = subspace[:, indices] @ U
# apply_smart_gauge(subspace)     

### 1. We know all states in the system

In [ ]:
%%time

# ev, evec - all eigenvalues and eigenstates of H0
# indices - states of the group A
indices = find_indices(ev, *window)

M0_1 = (evec[:, indices].T.conj() @ mat0 @ evec[:, indices]).diagonal().real
M1_1 = first_order(mat1, evec[:, indices])
M2_1 = second_order_explicit(mat1, ev, evec, indices)

### 2. We know only states we want to include in the effective model

In [ ]:
# %%time

# energies, subspace - k eigenvalues and eigenstates of H0
# indices - states of the group A
indices = find_indices(energies, *window)

M0_2 = (subspace[:, indices].T.conj() @ mat0 @ subspace[:, indices]).diagonal().real
M1_2 = first_order(mat1, subspace[:, indices])
M2_2 = second_order_kpm(mat0, mat1, energies[indices], subspace[:, indices])

### 3. We know states we want to include and some close states

In [ ]:
%%time

# energies, subspace - k eigenvalues and eigenstates of H0
# indices - states of the group A
indices = find_indices(energies, *window)

M0_3 = (subspace[:, indices].T.conj() @ mat0 @ subspace[:, indices]).diagonal().real
M1_3 = first_order(mat1, subspace[:, indices])
M2_3_exp = second_order_explicit(mat1, energies, subspace, indices)
M2_3_kpm = second_order_kpm(mat0, mat1, energies, subspace, indices)

# Look into the effective models

In [ ]:
smp_1 = sympify_perturbation(M0_1, components=[M1_1, M2_1], decimals=4)
smp_2 = sympify_perturbation(M0_2, components=[M1_2, M2_2], decimals=4)
smp_3 = sympify_perturbation(M0_3, components=[M1_3, M2_3_exp, M2_3_kpm], decimals=4)

In [ ]:
smp_1

In [ ]:
smp_2

In [ ]:
smp_3

## compare with exact solution

In [ ]:
def calc_energy(model, k):
    p = {'k_x': k, 'k_y': 0}
    mat = model(**p)
    return la.eigvalsh(mat)


model = kwant.continuum.lambdify(smp_1)
energies_eff_1 = np.array([calc_energy(model, k) for k in momenta])

model = kwant.continuum.lambdify(smp_2)
energies_eff_2 = np.array([calc_energy(model, k) for k in momenta])

model = kwant.continuum.lambdify(smp_3)
energies_eff_3 = np.array([calc_energy(model, k) for k in momenta])

In [ ]:
plt.plot(momenta, 1000 * energies_exact, 'k-');
plt.plot(momenta, 1000 * energies_eff_1, 'r-');
plt.plot(momenta, 1000 * energies_eff_2, 'g--');
plt.plot(momenta, 1000 * energies_eff_3, 'b-.');

plt.plot(xlims, 1000 * np.array([window, window]))
plt.ylim(*ylims)
plt.xlim(*xlims)

# check convergence

In [ ]:
def difference_kpm(kpm):
    M2 = M2_1
    
    assert set(M2) == set(kpm)
    output = 0
    for key, val in M2.items():
        output += la.norm(val - kpm[key])
    return output
    
    
def difference_mixed(exp, kpm):
    M2 = M2_1
    assert set(M2) == set(kpm)
    assert set(exp) == set(kpm)
    
    output = 0
    for key, val in M2.items():
        output += la.norm(val - kpm[key] - exp[key])
    return output
    

In [ ]:
moments = range(1000, 10000, 2000)


kpm_ds = []
mixed_ds = []

indices = find_indices(energies, *window)
mixed_exp = second_order_explicit(mat1, energies, subspace, indices)

for num_moments in moments:
    print(num_moments)
    
    kpm = second_order_kpm(mat0, mat1, energies[indices], subspace[:, indices], num_moments=num_moments)    
    mixed_kpm = second_order_kpm(mat0, mat1, energies, subspace, indices, num_moments=num_moments)
    
    kpm_ds.append(difference_kpm(kpm))
    mixed_ds.append(difference_mixed(mixed_exp, mixed_kpm))
    

In [ ]:
plt.plot(moments, kpm_ds, 'ro-', label='kpm')
plt.plot(moments, mixed_ds, 'bo-', label='mixed')
plt.ylabel('|explicit - kpm|')
plt.xlabel('# moments')
plt.ylim(1e-6, 1)
plt.yscale('log')
plt.grid()
plt.legend()